In [53]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import copy

import pyproj
print(pyproj.datadir.get_data_dir())
pyproj.datadir.set_data_dir('/usr/local/anaconda3/envs/info615/share/proj')

/usr/local/anaconda3/envs/info615/share/proj


In [25]:
#Load census data
df = pd.read_csv('dc_example/data/raw_data/includeshispanic_DECENNIALPL2020.P2_2022-03-09T103623/DECENNIALPL2020.P2_data_with_overlays_2022-01-07T105823.csv')
df.drop(0, inplace = True)

#Split out relevant part of GEOID
df['GEOID20'] = df['GEO_ID'].str.split('US').str[-1]

#Convert to numeric
cols_numeric = [col for col in df.columns if col not in ['GEO_ID','GEOID20', 'NAME']]
df[cols_numeric] = df[cols_numeric].apply(pd.to_numeric)

df.head()


,GEO_ID,NAME,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,P2_065N,P2_066N,P2_067N,P2_068N,P2_069N,P2_070N,P2_071N,P2_072N,P2_073N,GEOID20
1,1000000US110010001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...",607,66,541,528,459,24,0,42,...,0,0,0,0,0,0,0,0,0,110010001011000
2,1000000US110010001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...",282,22,260,248,205,18,0,25,...,0,0,0,0,0,0,0,0,0,110010001011001
3,1000000US110010001011002,"Block 1002, Block Group 1, Census Tract 1.01, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,110010001011002
4,1000000US110010001011003,"Block 1003, Block Group 1, Census Tract 1.01, ...",85,12,73,68,58,0,2,8,...,0,0,0,0,0,0,0,0,0,110010001011003
5,1000000US110010001011004,"Block 1004, Block Group 1, Census Tract 1.01, ...",56,7,49,44,40,2,0,1,...,0,0,0,0,0,0,0,0,0,110010001011004


In [26]:
#Load block shapefiles
block_gdf = gpd.read_file('dc_example/data/raw_data/tl_2020_11_tabblock20/tl_2020_11_tabblock20.shp')
block_gdf

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry
0,11,001,008002,2006,110010080022006,Block 2006,G5040,None,None,None,S,14524,0,+38.8927924,-076.9859904,82,152,"POLYGON ((-76.98648 38.89357, -76.98638 38.893..."
1,11,001,008002,2003,110010080022003,Block 2003,G5040,None,None,None,S,7884,0,+38.8930433,-076.9830792,18,45,"POLYGON ((-76.98365 38.89350, -76.98357 38.893..."
2,11,001,009907,2001,110010099072001,Block 2001,G5040,None,None,None,S,32579,0,+38.8826154,-076.9366848,150,300,"POLYGON ((-76.93936 38.88223, -76.93911 38.882..."
3,11,001,002201,2015,110010022012015,Block 2015,G5040,None,None,None,S,19590,0,+38.9468306,-077.0203939,54,153,"POLYGON ((-77.02152 38.94686, -77.02146 38.946..."
4,11,001,003301,3003,110010033013003,Block 3003,G5040,None,None,None,S,14227,0,+38.9186439,-077.0128448,86,198,"POLYGON ((-77.01354 38.91918, -77.01276 38.919..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,11,001,980000,1105,110019800001105,Block 1105,G5040,None,None,None,S,18993,0,+38.8954304,-077.0539974,0,0,"POLYGON ((-77.05542 38.89315, -77.05523 38.893..."
6008,11,001,002101,2003,110010021012003,Block 2003,G5040,None,None,None,S,24988,0,+38.9558782,-077.0210753,116,247,"POLYGON ((-77.02231 38.95638, -77.02157 38.956..."
6009,11,001,005601,2005,110010056012005,Block 2005,G5040,None,None,None,S,3747,719,+38.9029134,-077.0571337,0,4,"POLYGON ((-77.05756 38.90275, -77.05738 38.902..."
6010,11,001,009400,3008,110010094003008,Block 3008,G5040,None,None,None,S,19245,0,+38.9345903,-076.9711522,32,78,"POLYGON ((-76.97217 38.93417, -76.97217 38.934..."


In [27]:
#Merge
race_gdf = block_gdf.merge(df, on = 'GEOID20', how = 'right')

#Check - should return nothing
race_gdf[race_gdf['geometry'].isna()]

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P2_064N,P2_065N,P2_066N,P2_067N,P2_068N,P2_069N,P2_070N,P2_071N,P2_072N,P2_073N


In [28]:
#Load dictionary to group columns into racial groups
race_dict = pd.read_csv('dc_example/data/raw_data/includeshispanic_DECENNIALPL2020.P2_2022-03-09T103623/DECENNIALPL2020.P2_metadata_2022-01-07T105823_GROUPED.csv')

#Subset the groups we want to plot
race_groups_plot = ['hispanic', 'nh_white', 'nh_black', 'nh_asian_pacific', 'nh_native', 'nh_other', 'nh_multiple']
race_dict_sub = race_dict[race_dict['race_group'].isin(race_groups_plot)]
race_dict_sub

,var_name,var_description,race_group
1,P2_002N,!!Total:!!Hispanic or Latino,hispanic
4,P2_005N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_white
5,P2_006N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_black
6,P2_007N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_native
7,P2_008N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_asian_pacific
8,P2_009N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_asian_pacific
9,P2_010N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_other
10,P2_011N,!!Total:!!Not Hispanic or Latino:!!Population...,nh_multiple


In [29]:
#Merge in racial groups
race_agg_gdf = race_gdf[['GEOID20', 'NAME', 'geometry']]
race_grouped = race_dict_sub.groupby('race_group')
for race_group, group_df in race_grouped:
    cols_extract = list(group_df['var_name'].values)
    race_sub_gdf = race_gdf[cols_extract]
    
    #Sum (in case more than one column in race group)
    race_agg_gdf[race_group] = race_sub_gdf[cols_extract].sum(axis = 1)

race_agg_gdf

/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,GEOID20,NAME,geometry,hispanic,nh_asian_pacific,nh_black,nh_multiple,nh_native,nh_other,nh_white
0,110010001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05515 38.91022, -77.05501 38.910...",66,42,24,13,0,3,459
1,110010001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05714 38.91054, -77.05702 38.910...",22,25,18,12,0,0,205
2,110010001011002,"Block 1002, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05564 38.90539, -77.05556 38.905...",0,0,0,0,0,0,0
3,110010001011003,"Block 1003, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05704 38.90609, -77.05695 38.906...",12,8,0,5,2,0,58
4,110010001011004,"Block 1004, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05584 38.90896, -77.05583 38.908...",7,2,2,5,0,0,40
...,...,...,...,...,...,...,...,...,...,...
6007,110019800001193,"Block 1193, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.01990 38.89276, -77.01859 38.892...",0,0,0,0,0,0,0
6008,110019800001194,"Block 1194, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.05872 38.88616, -77.05849 38.886...",0,0,0,0,0,0,0
6009,110019800001195,"Block 1195, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.01198 38.88603, -77.01166 38.886...",0,0,0,0,0,0,0
6010,110019800001196,"Block 1196, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.01062 38.88507, -77.01050 38.885...",0,0,0,0,0,0,0


In [30]:
#Sum the number of points I need in each polygon
race_agg_gdf['total_points'] = race_agg_gdf[race_groups_plot].sum(axis = 1)
race_agg_gdf.head()

/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,GEOID20,NAME,geometry,hispanic,nh_asian_pacific,nh_black,nh_multiple,nh_native,nh_other,nh_white,total_points
0,110010001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05515 38.91022, -77.05501 38.910...",66,42,24,13,0,3,459,607
1,110010001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05714 38.91054, -77.05702 38.910...",22,25,18,12,0,0,205,282
2,110010001011002,"Block 1002, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05564 38.90539, -77.05556 38.905...",0,0,0,0,0,0,0,0
3,110010001011003,"Block 1003, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05704 38.90609, -77.05695 38.906...",12,8,0,5,2,0,58,85
4,110010001011004,"Block 1004, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05584 38.90896, -77.05583 38.908...",7,2,2,5,0,0,40,56


In [31]:
#Generate one point to represent one person within each block
def random_point_in_shp(shp):
    within = False
    while not within:
        x = np.random.uniform(shp.bounds[0], shp.bounds[2])
        y = np.random.uniform(shp.bounds[1], shp.bounds[3])
        within = shp.contains(Point(x, y))
        

    return [x,y]

In [32]:
#Looks like duplciates never happen - could comment out that part of task definition to speed up

In [33]:
def task_definition(row):
    #Counter
    row_num = row['index']
    if row_num % 500 == 0:
        print(row_num)
        
    #Operation
    points = []
    num_points = row['total_points']
    for i in range(num_points):
        new_point = random_point_in_shp(row['geometry'])
        points.append(new_point)
        
    points = np.array(points)
    possibledups = True
    while possibledups:
        reduced_points, indices_non_duplicate = np.unique(points, axis = 0, return_index = True)
        num_duplicates = points.shape[0] - reduced_points.shape[0]
        points = reduced_points
        if num_duplicates != 0:
            for i in range(num_duplicates):
                print('found duplicates retrying')
                new_point = random_point_in_shp(row['geometry'])
                points.append(new_point)
        else:
            possibledups = False
    return points

In [34]:
race_nonzero_agg_gdf = race_agg_gdf[race_agg_gdf['total_points'] != 0]
race_nonzero_agg_gdf.reset_index(inplace = True)

In [35]:
race_nonzero_agg_gdf['points'] = race_nonzero_agg_gdf.apply(lambda row: task_definition(row), axis = 1)

0
500
1000
1500
2000
3500
4000
5500


/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [36]:
#Check
race_nonzero_agg_gdf['len_points'] = race_nonzero_agg_gdf.apply(lambda row: np.array(row['points']).shape[0], axis = 1)
race_nonzero_agg_gdf

/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,index,GEOID20,NAME,geometry,hispanic,nh_asian_pacific,nh_black,nh_multiple,nh_native,nh_other,nh_white,total_points,points,len_points
0,0,110010001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05515 38.91022, -77.05501 38.910...",66,42,24,13,0,3,459,607,"[[-77.05504581231462, 38.91040327361093], [-77...",607
1,1,110010001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05714 38.91054, -77.05702 38.910...",22,25,18,12,0,0,205,282,"[[-77.05711267357546, 38.91000138715164], [-77...",282
2,3,110010001011003,"Block 1003, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05704 38.90609, -77.05695 38.906...",12,8,0,5,2,0,58,85,"[[-77.05685226892145, 38.90572878805476], [-77...",85
3,4,110010001011004,"Block 1004, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05584 38.90896, -77.05583 38.908...",7,2,2,5,0,0,40,56,"[[-77.05579679534353, 38.90887638474293], [-77...",56
4,5,110010001011005,"Block 1005, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05709 38.90852, -77.05678 38.908...",6,6,0,3,0,0,74,89,"[[-77.0570781038902, 38.908518867762005], [-77...",89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,5869,110019800001055,"Block 1055, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.01401 38.88852, -77.01398 38.888...",0,0,0,3,0,0,0,3,"[[-77.01386363349677, 38.88809066025994], [-77...",3
4496,5871,110019800001057,"Block 1057, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.00992 38.88799, -77.00906 38.887...",1,0,0,1,0,0,2,4,"[[-77.00987601399383, 38.88795074130877], [-77...",4
4497,5884,110019800001070,"Block 1070, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.00816 38.88760, -77.00812 38.887...",0,2,0,0,0,0,2,4,"[[-77.00773841525691, 38.88745411568015], [-77...",4
4498,5904,110019800001090,"Block 1090, Block Group 1, Census Tract 9800, ...","POLYGON ((-77.05016 38.89021, -77.05008 38.890...",0,0,5,0,0,0,3,8,"[[-77.04796204053658, 38.89112966494962], [-77...",8


In [46]:
arr = np.arange(9).reshape((3, 3))
print(arr)
np.random.shuffle(arr)
arr

[[0 1 2]
 [3 4 5]
 [6 7 8]]


array([[3, 4, 5],
       [6, 7, 8],
       [0, 1, 2]])

In [59]:
def shuffle_points(row):
    points_list = row['points']
    #     print('a',points_list[:2])
    points_list_return = copy.deepcopy(points_list)
    np.random.shuffle(points_list_return)
#     print('b', points_list_return[:2])
    return points_list_return
    
race_nonzero_agg_gdf['points_shuffled'] = race_nonzero_agg_gdf.apply(lambda row: shuffle_points(row), axis = 1)

Block 1000, Block Group 1, Census Tract 1.01, District of Columbia, District of Columbia
607
a [[-77.05260717  38.90998287]
 [-77.05301375  38.90994446]]
b [[-77.0545098   38.90947735]
 [-77.05453097  38.90968735]]


AttributeError: 'numpy.ndarray' object has no attribute 'shuffle'

In [56]:
race_nonzero_agg_gdf.head(2)

,index,GEOID20,NAME,geometry,hispanic,nh_asian_pacific,nh_black,nh_multiple,nh_native,nh_other,nh_white,total_points,points,len_points,points_shuffled
0,0,110010001011000,"Block 1000, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05515 38.91022, -77.05501 38.910...",66,42,24,13,0,3,459,607,"[[-77.05125020825244, 38.91007739782984], [-77...",607,"[[-77.05125020825244, 38.91007739782984], [-77..."
1,1,110010001011001,"Block 1001, Block Group 1, Census Tract 1.01, ...","POLYGON ((-77.05714 38.91054, -77.05702 38.910...",22,25,18,12,0,0,205,282,"[[-77.05591297844443, 38.910223254157266], [-7...",282,"[[-77.05591297844443, 38.910223254157266], [-7..."


In [63]:
def reshape(row, race_groups_plot):
    row_num = row['index']
    if row_num % 100 == 0:
        print(row_num)
    
    container_df = pd.DataFrame()
    i = 0
    points = row['points']
    np.random.shuffle(points) #Shuffle the points so they don't show up in lines
    points = [Point(point) for point in points]
    for race in race_groups_plot:
        num_points_race = int(row[race])
#         print(num_points_race)
        points_for_race = points[i:num_points_race+i]
#         print(len(points_for_race))
#         print(len([race]*num_points_race))
        i += num_points_race
#         print(i)
        race_df = pd.DataFrame({'race':[race]*num_points_race, 'geometry':points_for_race})
        container_df = container_df.append(race_df, ignore_index = True)
#         display(container_df)
        
    return container_df
        

In [64]:
#Reshape so that there is one row for each person
race_nonzero_agg_gdf['container_df'] = race_nonzero_agg_gdf.apply(lambda row: reshape(row, race_groups_plot), axis = 1)

0


/var/folders/x1/y94l059d54s9xkg8km690fd80000gn/T/ipykernel_73589/153083155.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(race_df, ignore_index = True)


100
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1900
2000
2100
2200
2400
2700
2800
3300
3400
3500
3700
3800
3900
4000
4200
4300
4600
4700
4800
5100
5200
5300
5400
5500
5600
5800


/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [2]:
race_nonzero_agg_gdf.loc[0,'container_df']

NameError: name 'race_nonzero_agg_gdf' is not defined

In [66]:
dc_one_point_one_person_df = pd.concat(race_nonzero_agg_gdf['container_df'].values)
dc_one_point_one_person_gdf = gpd.GeoDataFrame(dc_one_point_one_person_df, geometry='geometry')
dc_one_point_one_person_gdf = dc_one_point_one_person_gdf.set_crs(epsg = 4326)

In [67]:
dc_one_point_one_person_gdf.to_file('dc_example/data/processed_data/dc_one_point_one_person_2020.shp')

/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [68]:
grouped = dc_one_point_one_person_gdf.groupby('race')
for group, group_df in grouped:
    group_df.to_file('dc_example/data/processed_data/dc_one_point_one_person_2020_{}.shp'.format(group))

/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/usr/local/anaconda3/envs/info615/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropri

### testing out numpy

In [152]:
a = np.array([[1.1, 2.2], [1.1, 2.2], [3.4, 5], [1,2]])

In [154]:
pd.DataFrame([1,2,3])

,0
0,1
1,2
2,3


In [128]:
a.shape

(4, 2)

In [129]:
un, count = np.unique(a, axis = 0, return_index = True)
count

array([3, 0, 2])

In [130]:
np.delete(a, count, axis = 0)

array([[1.1, 2.2]])

In [131]:
a[count]

array([[1. , 2. ],
       [1.1, 2.2],
       [3.4, 5. ]])